In [1]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from collections import Counter
import random

data = load_breast_cancer()
X, y = data.data, data.target

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

print(f"Train set size: {len(X_train)} samples (70%)")
print(f"Validation set size: {len(X_val)} samples (15%)")
print(f"Test set size: {len(X_test)} samples (15%)")

Train set size: 398 samples (70%)
Validation set size: 85 samples (15%)
Test set size: 86 samples (15%)


In [2]:
def entropy(y):
    if len(y) == 0:
        return 0
    # Count each class
    counts = Counter(y)
    probabilities = [count / len(y) for count in counts.values()]
    # Entropy formula: -summation(p * log2(p))
    entropy_value = -np.sum([p * np.log2(p) for p in probabilities if p > 0])
    return entropy_value

def information_gain(y_parent, y_left, y_right):
    p = len(y_left) / len(y_parent)
    # IG = H(parent) - [weighted average of children's entropy]
    return entropy(y_parent) - (p * entropy(y_left) + (1 - p) * entropy(y_right))

def majority_vote(y):
    # Find the most common element in y
    return Counter(y).most_common(1)[0][0]

In [3]:
class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right

        # Leaf Node attribute (used for prediction)
        self.value = value

    # Check if this node is a leaf
    def is_leaf(self):
        return self.value is not None

In [4]:
class DecisionTreeClassifier:
    def __init__(self, max_depth=10, min_samples_split=2, max_features=None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features 
        self.root = None

    def fit(self, X, y):
        n_features = X.shape[1]

        # Determine features to consider at each split
        if self.max_features is None:
            self.feature_indices = np.arange(n_features) # All features
        elif isinstance(self.max_features, int):
            self.feature_indices = np.arange(n_features) # Will subsample later
        elif isinstance(self.max_features, float):
             # uses ratio for max features
            self.feature_indices = np.arange(n_features)
        else:
             self.feature_indices = np.arange(n_features)


        self.root = self._grow_tree(X, y, 0)
        return self

    def _grow_tree(self, X, y, depth):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        # Stopping conditions
        # 1-Reached max depth
        # 2-Samples less than minimum samples to split
        # 3-Pure node
        if (depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split):
            leaf_value = majority_vote(y)
            return Node(value=leaf_value)

        # Find best split
        # If max_features is set we select a random subset of features
        if self.max_features is None:
            feat_indices = self.feature_indices # Use all features
        else:
            # Randomly sample features for this split
            k = int(self.max_features) if isinstance(self.max_features, int) else int(np.sqrt(n_features))
            feat_indices = random.sample(list(self.feature_indices), k)
        
        best_feat, best_thresh = self._best_split(X, y, feat_indices)

        # If no gain found then this is a leaf node
        if best_feat is None:
            return Node(value=majority_vote(y))

        # --- Split Data ---
        left_indices = X[:, best_feat] <= best_thresh
        right_indices = X[:, best_feat] > best_thresh

        X_left, y_left = X[left_indices], y[left_indices]
        X_right, y_right = X[right_indices], y[right_indices]

        # --- Recursive Call ---
        left_child = self._grow_tree(X_left, y_left, depth + 1)
        right_child = self._grow_tree(X_right, y_right, depth + 1)

        return Node(best_feat, best_thresh, left_child, right_child)


    def _best_split(self, X, y, feature_indices):
        max_gain = -1
        best_feat, best_thresh = None, None

        for feat_idx in feature_indices:
            # Use sorted distinct values as candidate thresholds
            thresholds = np.unique(X[:, feat_idx])

            for thresh in thresholds:
                # Split on the threshold: x_j <= t and x_j > t
                left_indices = X[:, feat_idx] <= thresh
                right_indices = X[:, feat_idx] > thresh

                y_left, y_right = y[left_indices], y[right_indices]

                # Ensure split is valid (not empty on one side)
                if len(y_left) == 0 or len(y_right) == 0:
                    continue

                # Compute Information Gain
                current_gain = information_gain(y, y_left, y_right)

                # Update best split if gain is higher
                if current_gain > max_gain:
                    max_gain = current_gain
                    best_feat = feat_idx
                    best_thresh = thresh

        return best_feat, best_thresh

    def predict(self, X):
        #Returns predictions for a list of samples
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf():
            return node.value

        # Check the condition at the internal node
        if x[node.feature_index] <= node.threshold:
            return self._traverse_tree(x, node.left)
        else:
            return self._traverse_tree(x, node.right)

In [5]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def train_and_evaluate(X_train, y_train, X_val, y_val, X_test, y_test):
    max_depths = [2, 4, 6, 8, 10]
    min_samples_splits = [2, 5, 10]
    
    best_accuracy = -1
    best_params = {}
    validation_results = []

    for depth in max_depths:
        for min_samples in min_samples_splits:
            model = DecisionTreeClassifier(
                max_depth=depth, 
                min_samples_split=min_samples, 
                max_features=None
            ).fit(X_train, y_train)

            y_val_pred = model.predict(X_val)
            val_acc = accuracy_score(y_val, y_val_pred)

            validation_results.append({
                'max_depth': depth,
                'min_samples_split': min_samples,
                'validation_accuracy': val_acc
            })
            
            if val_acc > best_accuracy:
                best_accuracy = val_acc
                best_params = {'max_depth': depth, 'min_samples_split': min_samples}

    print(f"Best parameters: {best_params} with Val Accuracy: {best_accuracy:.4f}")
    
    X_train_val = np.vstack((X_train, X_val))
    y_train_val = np.hstack((y_train, y_val))
    
    final_model = DecisionTreeClassifier(
        max_depth=best_params['max_depth'], 
        min_samples_split=best_params['min_samples_split'],
        max_features=None
    ).fit(X_train_val, y_train_val) # Retrain on training + validation

    y_test_pred = final_model.predict(X_test)
    
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_precision = precision_score(y_test, y_test_pred, average=None) # Precision for both classes
    test_recall = recall_score(y_test, y_test_pred, average=None)       # Recall for both classes
    test_f1 = f1_score(y_test, y_test_pred, average=None)              # F1-score for both classes
    conf_matrix = confusion_matrix(y_test, y_test_pred)                # Confusion Matrix

    print("\nEvaluation Metrics:\n")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Class 0 (Malignant) - Precision: {test_precision[0]:.4f}, Recall: {test_recall[0]:.4f}, F1: {test_f1[0]:.4f}")
    print(f"Class 1 (Benign) - Precision: {test_precision[1]:.4f}, Recall: {test_recall[1]:.4f}, F1: {test_f1[1]:.4f}")
    print("\nConfusion Matrix:\n", conf_matrix)
    
    return validation_results, final_model

validation_history, final_dt_model = train_and_evaluate(X_train, y_train, X_val, y_val, X_test, y_test)

Best parameters: {'max_depth': 4, 'min_samples_split': 2} with Val Accuracy: 0.9765

Evaluation Metrics:

Test Accuracy: 0.8837
Class 0 (Malignant) - Precision: 0.8667, Recall: 0.8125, F1: 0.8387
Class 1 (Benign) - Precision: 0.8929, Recall: 0.9259, F1: 0.9091

Confusion Matrix:
 [[26  6]
 [ 4 50]]
